# Imports

In [2]:
import numpy as np
import import_ipynb
import _pickle as pkl
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
from sklearn.datasets import make_blobs, make_circles, make_moons, make_classification
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from utils import plot
from utils.samples_generator import make_spiral, make_square, make_cubic, make_exp, make_log10

# Funções de Ativação

In [3]:
def linear(x, derivative=False):
    return np.ones_like(x) if derivative else x

def sigmoid(x, derivative=False):
    if derivative:
        y = sigmoid(x)
        return y*(1 - y)
    return 1.0/(1.0 + np.exp(-x))

def tanh(x, derivative=False):
    if derivative:
        y = tanh(x)
        return 1 - y**2
    return (np.exp(x) - np.exp(-x))/(np.exp(x) + np.exp(-x))

def relu(x, derivative=False):
    if derivative:
        return np.where(x <= 0, 0, 1)
    return np.maximum(0, x)

def leaky_relu(x, derivative=False):
    alpha = 0.1
    if derivative:
        return np.where(x <= 0, alpha, 1)
    return np.where(x <= 0, alpha*x, x)

def elu(x, derivative=False):
    alpha = 1.0
    if derivative:
        y = elu(x)
        return np.where(x <= 0, y + alpha, 1)
    return np.where(x <= 0, alpha*(np.exp(x) - 1), x)

# Funções Auxiliares

In [4]:
def softmax(x, y_oh=None, derivative=False):
    if derivative:
        y_pred = softmax(x)
        k = np.nonzero(y_pred * y_oh)
        pk = y_pred[k]
        y_pred[k] = pk * (1.0 - pk)
        return y_pred
    exp = np.exp(x)
    return exp / np.sum(exp, axis=1, keepdims=True)

# Funções de Custo

In [5]:
def mae(y, y_pred, derivative=False):
    
    if(derivative):
        return np.where(y_pred>y, 1, -1) / y.shape[0]
    
    return np.mean(np.abs(y - y_pred))

def mse(y, y_pred, derivative = False):
    
    if(derivative):
        return -(y - y_pred) / y.shape[0]
    
    return 0.5 * np.mean((y - y_pred)**2)

### Para Classificação Binaria

In [6]:
def binary_cross_entropy ( y, y_pred, derivative = False):
    if derivative:
        return -(y - y_pred) / (y_pred * (1 - y_pred) *y.shape[0])
    return -np.mean(y*np.log(y_pred) + (1 - y)*np.log(1-y_pred))

def sigmoid_cross_entropy(y, y_pred, derivative=False):
    y_sigmoid = sidmoid(y_pred)
    
    if derivative:
        return -(y - y_sigmoid) / y.shape[0]
    
    return -np.mean(y*np.log(y_sigmoid) + (1 - y)*np.log(1 - y_sigmoid))


### Para Classificação Multiclasse

In [7]:
def neg_log_likelihood(y_oh, y_pred, derivative=False):
    k = np.nonzero(y_pred * y_oh)
    pk = y_pred[k]
    if derivative:
        y_pred[k] = (-1.0 / pk)
        return y_pred
    return np.mean(-np.log(pk))

def softmax_neg_log_likelihood(y_oh, y_pred, derivative=False):
    y_softmax = softmax(y_pred)
    if derivative:
        return -(y_oh - y_softmax) / y_oh.shape[0]    
    return neg_log_likelihood(y_oh, y_softmax)

# Neural Network

In [8]:
class Layer():
    def __init__(self, input_dim, output_dim, activation=linear):
        self.input = None
        self.weights = np.random.randn(output_dim, input_dim)
        self.biases = np.random.randn(1, output_dim)
        self.activation = activation
        
        self._activ_inp, self._activ_out = None, None
        
        self._dweights, self._dbiases = None, None

class NeuralNetwork():
        def __init__(self, cost_func=mse, learning_rate=1e-3):
            self.layers = []
            self.cost_func = cost_func
            self.learning_rate = learning_rate
        
        def fit(self, x_train, y_train, epochs=100, verbose=10):
            for epoch in range( epochs + 1):
                y_pred = self.__feedforward(x_train)
                self.__backprop(y_train, y_pred)
                
                if(epoch % verbose == 0):
                    loss_train = self.cost_func(y_train, self.predict(x_train))
                    print(f"epoch: {epoch:=4}/{epochs:=4} loss_train: {loss_train:.8f}")
        
        def predict(self, x):
            return self.__feedforward(x)
            
        
        def __feedforward(self, x):
            self.layers[0].input = x
            
            for current_layer, next_layer in zip(self.layers, self.layers[1:] + [Layer(0,0)]):
                y = np.dot(current_layer.input, current_layer.weights.T) + current_layer.biases
                current_layer._activ_inp = y
                current_layer._activ_out = next_layer.input = current_layer.activation(y)
            
            return self.layers[-1]._activ_out
                

        def __backprop(self, y, y_pred):
            last_delta = self.cost_func(y, y_pred, derivative=True)
            for layer in reversed(self.layers):
                dactivation = layer.activation(layer._activ_inp, derivative=True) * last_delta
                last_delta = np.dot(dactivation,layer.weights)
                layer._dweights = np.dot(dactivation.T, layer.input)
                layer._dbiases = 1.0 * dactivation.sum(axis=0, keepdims=True)
                
            for layer in reversed (self.layers):
                layer.weights = layer.weights - self.learning_rate * layer._dweights
                layer.biases = layer.biases - self.learning_rate * layer._dbiases